### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
## Split the data into training,validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [4]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8], shape=(3673,))

In [5]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [10]:
train_x

array([[ 5.8 ,  0.28,  0.35, ...,  3.28,  0.5 , 10.2 ],
       [ 7.1 ,  0.21,  0.28, ...,  3.35,  0.64, 10.2 ],
       [ 5.9 ,  0.32,  0.26, ...,  3.24,  0.36, 10.7 ],
       ...,
       [ 7.4 ,  0.2 ,  0.37, ...,  3.14,  0.61, 11.8 ],
       [ 8.1 ,  0.3 ,  0.31, ...,  2.99,  0.45, 11.1 ],
       [ 6.6 ,  0.36,  0.21, ...,  3.18,  0.41,  9.9 ]], shape=(2938, 11))

In [6]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [7]:
train_x.shape[1]

11

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3673 entries, 2835 to 860
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         3673 non-null   float64
 1   volatile acidity      3673 non-null   float64
 2   citric acid           3673 non-null   float64
 3   residual sugar        3673 non-null   float64
 4   chlorides             3673 non-null   float64
 5   free sulfur dioxide   3673 non-null   float64
 6   total sulfur dioxide  3673 non-null   float64
 7   density               3673 non-null   float64
 8   pH                    3673 non-null   float64
 9   sulphates             3673 non-null   float64
 10  alcohol               3673 non-null   float64
 11  quality               3673 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 373.0 KB


In [11]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [12]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [13]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [15]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 32s 717ms/step - loss: 39.2969 - root_mean_squared_error: 6.2687
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.2380 - root_mean_squared_error: 3.7952   
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.9121 - root_mean_squared_error: 3.1615
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.9761 - root_mean_squared_error: 2.8389 
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 3.3971 - root_mean_squared_error: 1.8431 - val_loss: 0.5587 - val_root_mean_squared_error: 0.7474

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.6542 - root_mean_squared_error: 0.8088
17/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6387 - root_mean_squared_error: 0.7991 
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6336 - root_mean_squared_error: 0.7960
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6048 - root_mean_squared_error: 0.7777 - val_loss: 0.5285 - val_ro

2025/09/17 23:10:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 26s 596ms/step - loss: 32.3034 - root_mean_squared_error: 5.6836
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.1280 - root_mean_squared_error: 5.4887   
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.1855 - root_mean_squared_error: 5.4940
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 30.2733 - root_mean_squared_error: 5.5021 - val_loss: 30.4761 - val_root_mean_squared_error: 5.5205

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 30.1283 - root_mean_squared_error: 5.4889
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 30.2966 - root_mean_squared_error: 5.5042 
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.0377 - root_mean_squared_error: 5.4806
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.9470 - root_mean_squared_error: 5.4723
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.9132 - ro

2025/09/17 23:10:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 34s 766ms/step - loss: 30.5637 - root_mean_squared_error: 5.5284
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26.7883 - root_mean_squared_error: 5.1667   
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 21.6528 - root_mean_squared_error: 4.6082
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19.5711 - root_mean_squared_error: 4.3662
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 11.1638 - root_mean_squared_error: 3.3412 - val_loss: 2.3082 - val_root_mean_squared_error: 1.5193

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 2.8251 - root_mean_squared_error: 1.6808
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.2953 - root_mean_squared_error: 1.5130 
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.4486 - root_mean_squared_error: 1.5622
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.7674 - root_m

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - loss: 38.9996 - root_mean_squared_error: 6.2450
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.1910 - root_mean_squared_error: 6.0982
40/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.5203 - root_mean_squared_error: 6.0428
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 35.1263 - root_mean_squared_error: 5.9267 - val_loss: 33.8277 - val_root_mean_squared_error: 5.8162

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 32.0775 - root_mean_squared_error: 5.6637
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.6237 - root_mean_squared_error: 5.7984 
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.1167 - root_mean_squared_error: 5.7544
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.9298 - root_mean_squared_error: 5.6507 - val_loss: 30.7325 - val_root_mean_squared_error: 5.5437

Epoch

2025/09/17 23:11:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [01:16<00:00, 19.21s/trial, best loss: 0.7673206925392151]

2025/09/17 23:11:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.039494615943415676), 'momentum': np.float64(0.8960743280258954)}
Best eval rmse: 0.7673206925392151


In [17]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/ccc08742bd7348ebabb473ba1f0e99de/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

f:\data science\udemy courses 1\mlops\MLFLOW_projects\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[6.016469 ],
       [6.2708416],
       [6.7584934],
       ...,
       [6.813589 ],
       [6.6253557],
       [6.2944694]], shape=(1225, 1), dtype=float32)

In [18]:
# Load model as a PyFuncModel.
model_uri = 'runs:/ccc08742bd7348ebabb473ba1f0e99de/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[6.016469 ],
       [6.2708416],
       [6.7584934],
       ...,
       [6.813589 ],
       [6.6253557],
       [6.2944694]], shape=(1225, 1), dtype=float32)